In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

def load_fer2013_data(csv_path, img_dir):
  # Load the CSV file that contains image data and labels
  df = pd.read_csv(csv_path)

  # Print the first few rows of the dataframe to inspect it
  print(df.head())

  # Initialize lists for image data and labels
  images = []
  labels = []

  # Iterate over each row in the dataframe
  for _, row in df.iterrows():
    # Extract the image data (comma-separated pixel values)
    pixels = np.array(row['pixels'].split(), dtype=np.uint8)
    # Reshape the pixels into an image (48x48 pixels, as per FER-2013)
    image = pixels.reshape(48, 48, 1)
    images.append(image)

    # Append the corresponding emotion label
    labels.append(int(row['emotion']))

  # Convert lists to numpy arrays
  X = np.array(images)
  y = np.array(labels)

  # Normalize image data to [0, 1]
  X = X.astype('float32') / 255.0

  # Split the dataset into training and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  print(f"Loaded {len(X)} images from the dataset.")
  return X_train, X_test, y_train, y_test
